# Base de dados do satelite kepler 

In [ ]:
!pip install lightkurve
import lightkurve as lk
import numpy as np
import pandas as pd
import json
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.4 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=adf4bfa93f11145b14c3b23ee1867753072ba325d96a60295abfdc9d5723a56f
  Stored in directory: /root/.cache/pip/wheels/f5/60/60/28df6c25f4d22b73d1a2b1c7e4842a5e2178e35e47b62e8e9a
  Created wheel for memoization: filename=me

## vamos rodar um script para pegar de Kepler-1 a Kepler-100
 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
with open("/content/gdrive/MyDrive/logArthur/kepler.csv","a") as f:
    f.write("sky_position,max_power,period_at_max_power,transit_time_at_max_power,duration_at_max_power"+"\n")

In [ ]:
df_teste = pd.read_csv("/content/gdrive/MyDrive/logArthur/kepler.csv")
df_teste.tail(15)

,sky_position,max_power,period_at_max_power,transit_time_at_max_power,duration_at_max_power
971,Kepler-986,1010.1406695450833,56.42664266426643 d,151.4972843576537,0.25 d
972,Kepler-987,2585.315380957427,396.52835283528356 d,184.212317405286,0.33 d
973,Kepler-988,3330.4680641549357,12.412541254125413 d,139.77247618623892,0.33 d
974,Kepler-989,2018.932625890448,300.00060006000604 d,169.7877178602561,0.33 d
975,Kepler-990,379.1944309850308,19.834683468346835 d,135.12677510721028,0.15 d
976,Kepler-991,469.84557446128,82.52385238523853 d,209.21668632900224,0.25 d
977,Kepler-992,70047.33250206134,26.17941794179418 d,143.10933512981893,0.33 d
978,Kepler-993,2974.2677705216083,10.936093609360936 d,358.61724873103844,0.33 d
979,Kepler-994,13975.840513494617,347.96519651965195 d,381.67726647800646,0.33 d
980,Kepler-995,65057.30827097571,18.3981398139814 d,140.92803342715254,0.33 d


In [ ]:
for i in range(795,1000):
    sky_position = "Kepler-" + str(i)
    try:
        search_result = lk.search_lightcurve(sky_position, author='Kepler', cadence='long')
        # Download all available Kepler light curves
        lc_collection = search_result.download_all()
        # Flatten the light curve
        lc = lc_collection.stitch().flatten(window_length=901).remove_outliers()
        # Create array of periods to search
        period = np.linspace(1, 400, 10000)
        # Create a BLSPeriodogram
        bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500);
        period_at_max_power = bls.period_at_max_power
        transit_time_at_max_power = bls.transit_time_at_max_power
        duration_at_max_power = bls.duration_at_max_power
        max_power = bls.max_power
        light_curve = np.array(lc.flux)
        np.savetxt(f"/content/gdrive/MyDrive/log_lc_flux/{sky_position}.csv", light_curve, delimiter=',')
        with open("/content/gdrive/MyDrive/logArthur/kepler.csv","a") as f:
            f.write(f"{sky_position}, {max_power}, {period_at_max_power}, {transit_time_at_max_power}, {duration_at_max_power}\n")
    except:
        with open("/content/gdrive/MyDrive/logArthur/missing_data_sky_position","a") as f:
            f.write(sky_position + "\n")

In [ ]:
lista = np.loadtxt("/content/gdrive/MyDrive/log_lc_flux/Kepler-999.csv")

print(len(lista))

65205
